In [1]:
import os
import numpy as np
import pandas as pd
from scipy.io import wavfile
from util import util

import torch
from beats.BEATs import BEATs, BEATsConfig

1 Physical GPUs, 1 Logical GPUs


In [2]:
df = pd.read_csv(os.path.join(util.dataPath, 'training datalist.csv'))
df['label'] = df['Disease category']
df['path'] = df['ID'].apply(lambda x: os.path.join(util.dataPath, 'training_voice_data', '{}.wav'.format(x)))
outfile = os.path.join(util.dataPath, 'training.pkl')
df

,ID,Sex,Age,Disease category,Narrow pitch range,Decreased volume,Fatigue,Dryness,Lumping,heartburn,...,Hypertension,CAD,Head and Neck Cancer,Head injury,CVA,Voice handicap index - 10,setName,foldIdx,label,path
0,1202f15,2,39,1,1,1,1,1,1,0,...,0,0,0,0,0,22.0,val,4,1,/outerDisk1/rawDatasets/multimodalPathological...
1,0600ve0,1,69,2,1,1,1,1,0,0,...,0,0,0,0,1,19.0,train,0,2,/outerDisk1/rawDatasets/multimodalPathological...
2,1001o7l,2,59,2,1,1,1,1,0,0,...,0,0,0,0,0,18.0,train,0,2,/outerDisk1/rawDatasets/multimodalPathological...
3,1201c1t,2,47,1,1,0,1,1,1,0,...,0,0,0,0,0,27.0,train,2,1,/outerDisk1/rawDatasets/multimodalPathological...
4,0402jvt,1,87,1,0,0,0,0,0,0,...,1,0,0,0,0,16.0,train,0,1,/outerDisk1/rawDatasets/multimodalPathological...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0G00ftn,1,75,3,0,0,0,0,0,0,...,1,1,0,0,0,21.0,train,1,3,/outerDisk1/rawDatasets/multimodalPathological...
996,1201pkr,1,40,4,0,0,1,0,0,0,...,0,0,0,0,0,21.0,train,0,4,/outerDisk1/rawDatasets/multimodalPathological...
997,0202p64,2,68,3,0,1,1,0,0,0,...,0,0,0,0,0,27.0,val,3,3,/outerDisk1/rawDatasets/multimodalPathological...
998,12021au,2,42,2,0,0,1,1,1,0,...,0,0,0,0,0,12.0,train,0,2,/outerDisk1/rawDatasets/multimodalPathological...


In [5]:
# public test
df = pd.read_csv(os.path.join(util.publicTestPath, 'test_datalist_public.csv'))
df['path'] = df['ID'].apply(lambda x: os.path.join(util.publicTestPath, 'test_data_public', '{}.wav'.format(x)))
outfile = os.path.join(util.publicTestPath, 'publitTest.pkl')
df

,ID,Sex,Age,Narrow pitch range,Decreased volume,Fatigue,Dryness,Lumping,heartburn,Choking,...,Noise at work,Occupational vocal demand,Diabetes,Hypertension,CAD,Head and Neck Cancer,Head injury,CVA,Voice handicap index - 10,path
0,1301skj,2,27,0,1,1,1,1,0,0,...,3,1,0,0,0,0,0,0,30.0,/outerDisk1/rawDatasets/multimodalPathological...
1,1100j0u,1,50,0,0,0,0,0,0,0,...,2,2,0,1,0,0,0,0,30.0,/outerDisk1/rawDatasets/multimodalPathological...
2,0G00r5c,1,41,1,1,0,0,0,0,0,...,2,3,0,0,0,0,0,0,29.0,/outerDisk1/rawDatasets/multimodalPathological...
3,1202v5m,2,48,1,0,0,1,1,0,0,...,1,4,0,0,0,0,0,0,5.0,/outerDisk1/rawDatasets/multimodalPathological...
4,0C01365,2,43,0,1,0,1,0,1,1,...,2,1,0,0,1,0,0,0,27.0,/outerDisk1/rawDatasets/multimodalPathological...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0C0147g,2,39,1,0,0,0,0,0,0,...,2,2,0,0,0,0,0,0,30.0,/outerDisk1/rawDatasets/multimodalPathological...
496,13021ib,2,58,0,0,1,1,0,1,0,...,3,1,0,0,0,0,0,0,30.0,/outerDisk1/rawDatasets/multimodalPathological...
497,110049h,1,72,0,1,1,0,0,0,0,...,1,3,0,1,0,0,0,0,32.0,/outerDisk1/rawDatasets/multimodalPathological...
498,0T01khc,2,60,1,0,0,0,1,0,0,...,2,4,0,0,0,1,0,0,3.0,/outerDisk1/rawDatasets/multimodalPathological...


In [6]:
# private test
df = pd.read_csv(os.path.join(util.privateTestPath, 'test_datalist_private.csv'))
df['path'] = df['ID'].apply(lambda x: os.path.join(util.privateTestPath, 'test_data_private', '{}.wav'.format(x)))
outfile = os.path.join(util.privateTestPath, 'privateTest.pkl')
df

,ID,Sex,Age,Narrow pitch range,Decreased volume,Fatigue,Dryness,Lumping,heartburn,Choking,...,Noise at work,Occupational vocal demand,Diabetes,Hypertension,CAD,Head and Neck Cancer,Head injury,CVA,Voice handicap index - 10,path
0,0U02g8u,2,77,0,1,0,1,0,0,0,...,1,3,1,1,0,0,0,1,14.0,/outerDisk1/rawDatasets/multimodalPathological...
1,0Y00jid,2,68,0,0,1,1,1,0,1,...,1,4,0,0,0,0,0,0,8.0,/outerDisk1/rawDatasets/multimodalPathological...
2,0R02296,1,38,0,1,0,1,1,0,0,...,2,1,0,0,0,1,0,0,25.0,/outerDisk1/rawDatasets/multimodalPathological...
3,0W016c9,1,58,0,1,1,0,1,0,0,...,1,3,0,1,0,0,0,0,34.0,/outerDisk1/rawDatasets/multimodalPathological...
4,11006gd,2,75,0,1,1,0,1,0,0,...,1,3,1,1,0,0,0,0,28.0,/outerDisk1/rawDatasets/multimodalPathological...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1001abu,2,29,1,1,1,0,0,0,0,...,2,1,0,0,0,0,0,0,39.0,/outerDisk1/rawDatasets/multimodalPathological...
496,1002ss4,2,77,1,1,0,0,0,0,0,...,1,2,0,0,0,0,0,0,21.0,/outerDisk1/rawDatasets/multimodalPathological...
497,0W011bi,1,44,0,1,1,1,1,1,0,...,1,1,0,0,0,0,0,0,27.0,/outerDisk1/rawDatasets/multimodalPathological...
498,0R00num,1,63,0,0,1,1,0,0,1,...,2,2,0,0,0,0,0,0,40.0,/outerDisk1/rawDatasets/multimodalPathological...


In [4]:
# load the pre-trained checkpoints
checkpoint = torch.load('BEATs_iter3_plus_AS2M.pt')

cfg = BEATsConfig(checkpoint['cfg'])
BEATs_model = BEATs(cfg)
BEATs_model.load_state_dict(checkpoint['model'])
BEATs_model.eval()

# extract the the audio representation
audio_input_16khz = torch.randn(1, 132300)
padding_mask = torch.zeros(1, 132300).bool()

representation = BEATs_model.extract_features(audio_input_16khz, padding_mask=padding_mask)[0]
representation

tensor([[[-0.0117, -0.5506, -0.7753,  ..., -0.0241,  0.2469, -0.0137],
         [ 0.2657, -0.2052, -0.2970,  ...,  0.1308,  0.6032, -0.1065],
         [ 0.6168, -0.3349,  0.1502,  ...,  0.0786,  0.0810,  0.0244],
         ...,
         [-0.1556, -0.0361,  0.6174,  ...,  0.0835, -0.1675,  0.1355],
         [-0.1156, -0.1233,  0.8208,  ..., -0.2111, -0.5564,  0.1147],
         [-0.1777, -0.4909,  0.3607,  ...,  0.1084, -0.4295,  0.2782]]],
       grad_fn=<TransposeBackward0>)

In [7]:
df['Age'] /= 100

onehotCols = ['label', 'Smoking', 'Drinking', 'Onset of dysphonia ', 'Diurnal pattern']
for col in onehotCols:
    if col in df:
        df[col] = pd.Series(pd.get_dummies(df[col]).values.tolist())
minmaxCols = ['Sex', 'PPD', 'frequency', 'Noise at work', 'Occupational vocal demand', 'Voice handicap index - 10']
for col in minmaxCols:
    df[col] = df[col].fillna(0)
    df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())

df['signal'] = df['path'].apply(lambda x: wavfile.read(x)[1] / 2**31)
maxSize = df['signal'].apply(lambda x: x.shape[0]).max()
df['signal'] = df['signal'].apply(lambda x: np.concatenate([x] * int(np.ceil(maxSize / x.shape[0])))[:maxSize])

df['beatsFeature'] = df['signal'].apply(lambda x: BEATs_model.extract_features(torch.from_numpy(np.expand_dims(x, 0).astype(np.float32)), padding_mask=torch.zeros(1, x.shape[0]).bool())[0].detach().numpy()[0])

del df['path']
df.to_pickle(outfile)

df

,ID,Sex,Age,Narrow pitch range,Decreased volume,Fatigue,Dryness,Lumping,heartburn,Choking,...,Occupational vocal demand,Diabetes,Hypertension,CAD,Head and Neck Cancer,Head injury,CVA,Voice handicap index - 10,signal,beatsFeature
0,0U02g8u,1.0,0.77,0,1,0,1,0,0,0,...,0.666667,1,1,0,0,0,1,0.350,"[-0.111175537109375, -0.11004638671875, -0.108...","[[0.15713872, 0.09611868, -0.027490275, 0.0898..."
1,0Y00jid,1.0,0.68,0,0,1,1,1,0,1,...,1.000000,0,0,0,0,0,0,0.200,"[0.04358541965484619, 0.04709649085998535, 0.0...","[[0.03280581, 0.42030498, 0.0077399346, 0.3362..."
2,0R02296,0.0,0.38,0,1,0,1,1,0,0,...,0.000000,0,0,0,1,0,0,0.625,"[-0.037384033203125, -0.037811279296875, -0.03...","[[-0.46499285, -0.07755125, -0.20460303, 0.180..."
3,0W016c9,0.0,0.58,0,1,1,0,1,0,0,...,0.666667,0,1,0,0,0,0,0.850,"[0.111572265625, 0.112396240234375, 0.11273193...","[[-0.14635128, 0.0033875378, -0.10318614, 0.02..."
4,11006gd,1.0,0.75,0,1,1,0,1,0,0,...,0.666667,1,1,0,0,0,0,0.700,"[-0.0194091796875, -0.014984130859375, -0.0310...","[[-0.2823963, 0.1606602, -0.22187842, -0.03156..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1001abu,1.0,0.29,1,1,1,0,0,0,0,...,0.000000,0,0,0,0,0,0,0.975,"[-0.0367431640625, -0.043182373046875, -0.0483...","[[-0.17591335, -0.048719455, -0.19188301, 0.17..."
496,1002ss4,1.0,0.77,1,1,0,0,0,0,0,...,0.333333,0,0,0,0,0,0,0.525,"[0.006805419921875, 0.007598876953125, 0.00637...","[[-0.22389068, -0.052827187, -0.25321302, 0.26..."
497,0W011bi,0.0,0.44,0,1,1,1,1,1,0,...,0.000000,0,0,0,0,0,0,0.675,"[-0.007049560546875, -0.0037841796875, -0.0007...","[[-0.29109988, -0.15586075, 0.010163356, 0.432..."
498,0R00num,0.0,0.63,0,0,1,1,0,0,1,...,0.333333,0,0,0,0,0,0,1.000,"[-0.1129150390625, -0.1322021484375, -0.134124...","[[0.12534869, -0.14788422, -0.22939701, -0.018..."


In [6]:
df = pd.read_pickle(os.path.join(util.dataPath, 'training.pkl'))
df.groupby("Disease category").count()['ID'] / len(df)

Disease category
1    0.536
2    0.220
3    0.168
4    0.044
5    0.032
Name: ID, dtype: float64